In [1]:
import pandas as pd

In [2]:
interactions_train = pd.read_csv('interactions_train.csv')
interactions_validation = pd.read_csv('interactions_validation.csv')
interactions_test = pd.read_csv('interactions_test.csv')
PP_users = pd.read_csv('PP_users.csv')
PP_recipes = pd.read_csv('PP_recipes.csv')
RAW_intreractions = pd.read_csv('RAW_interactions.csv')
RAW_recipes = pd.read_csv('RAW_recipes.csv')

In [3]:
interactions = pd.concat([interactions_train, interactions_validation, interactions_test])

In [4]:
RAW_recipes = RAW_recipes.drop(columns = ['contributor_id', 'submitted'])
RAW_recipes.rename(columns={'id':'recipe_id'})

,name,recipe_id,minutes,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients
0,arriba baked winter squash mexican style,137739,55,"['60-minutes-or-less', 'time-to-make', 'course...","[51.5, 0.0, 13.0, 0.0, 2.0, 0.0, 4.0]",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7
1,a bit different breakfast pizza,31490,30,"['30-minutes-or-less', 'time-to-make', 'course...","[173.4, 18.0, 0.0, 17.0, 22.0, 35.0, 1.0]",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'egg...",6
2,all in the kitchen chili,112140,130,"['time-to-make', 'course', 'preparation', 'mai...","[269.8, 22.0, 32.0, 48.0, 39.0, 27.0, 5.0]",6,"['brown ground beef in large pot', 'add choppe...",this modified version of 'mom's' chili was a h...,"['ground beef', 'yellow onions', 'diced tomato...",13
3,alouette potatoes,59389,45,"['60-minutes-or-less', 'time-to-make', 'course...","[368.1, 17.0, 10.0, 2.0, 14.0, 8.0, 20.0]",11,['place potatoes in a large pot of lightly sal...,"this is a super easy, great tasting, make ahea...","['spreadable cheese with garlic and herbs', 'n...",11
4,amish tomato ketchup for canning,44061,190,"['weeknight', 'time-to-make', 'course', 'main-...","[352.9, 1.0, 337.0, 23.0, 3.0, 0.0, 28.0]",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'sugar...",8
...,...,...,...,...,...,...,...,...,...,...
231632,zydeco soup,486161,60,"['ham', '60-minutes-or-less', 'time-to-make', ...","[415.2, 26.0, 34.0, 26.0, 44.0, 21.0, 15.0]",7,"['heat oil in a 4-quart dutch oven', 'add cele...",this is a delicious soup that i originally fou...,"['celery', 'onion', 'green sweet pepper', 'gar...",22
231633,zydeco spice mix,493372,5,"['15-minutes-or-less', 'time-to-make', 'course...","[14.8, 0.0, 2.0, 58.0, 1.0, 0.0, 1.0]",1,['mix all ingredients together thoroughly'],this spice mix will make your taste buds dance!,"['paprika', 'salt', 'garlic powder', 'onion po...",13
231634,zydeco ya ya deviled eggs,308080,40,"['60-minutes-or-less', 'time-to-make', 'course...","[59.2, 6.0, 2.0, 3.0, 6.0, 5.0, 0.0]",7,"['in a bowl , combine the mashed yolks and may...","deviled eggs, cajun-style","['hard-cooked eggs', 'mayonnaise', 'dijon must...",8
231635,cookies by design cookies on a stick,298512,29,"['30-minutes-or-less', 'time-to-make', 'course...","[188.0, 11.0, 57.0, 11.0, 7.0, 21.0, 9.0]",9,['place melted butter in a large mixing bowl a...,"i've heard of the 'cookies by design' company,...","['butter', 'eagle brand condensed milk', 'ligh...",10


In [5]:
interactions = interactions.merge(RAW_intreractions[['user_id', 'recipe_id', 'review']],
                            on=['user_id', 'recipe_id'],
                            how='left')

In [6]:
interactions = interactions.merge(RAW_recipes,
                                left_on='recipe_id',
                                right_on='id',
                                how='left')

interactions = interactions.drop(columns = ['id'])

In [7]:
interactions = interactions.merge(PP_recipes,
                                left_on='recipe_id',
                                right_on='id',
                                how='left')

interactions = interactions.drop(columns = ['id', 'name', 'ingredients', 'nutrition', 'steps', 'u', 'i_x', 'i_y', 'date', 'description', 'review', 'ingredient_tokens'])

In [8]:
import numpy as np

unique_tags = set(tag for sublist in interactions['tags'] for tag in sublist)
tag_to_index = {tag: idx for idx, tag in enumerate(unique_tags)}

def tags_to_vector(tags, tag_to_index):
    vector = np.zeros(len(tag_to_index))
    for tag in tags:
        if tag in tag_to_index:
            vector[tag_to_index[tag]] = 1
    return vector

interactions['tags_vector'] = interactions['tags'].apply(lambda x: tags_to_vector(x, tag_to_index))

interactions = interactions.drop(columns=['tags'])

In [9]:
tags_vectors = np.stack(interactions['tags_vector'].values)

tags_expanded = pd.DataFrame(tags_vectors, index=interactions.index)

interactions = pd.concat([interactions, tags_expanded], axis=1)

interactions.columns = list(interactions.columns[:len(interactions.columns) - 44]) + [f'tag_{i}' for i in range(1, 45)]


In [10]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

recipes = interactions.drop(columns=['user_id', 'rating', 'name_tokens', 'steps_tokens', 'techniques', 'ingredient_ids'])
recipes['tags_vector'] = recipes['tags_vector'].apply(lambda x: x.tolist())

tags_vectors = np.stack(recipes['tags_vector'].values)
tags_expanded = pd.DataFrame(tags_vectors, index=recipes.index)

recipes = pd.concat([recipes, tags_expanded], axis=1)

recipes.columns = list(recipes.columns[:len(recipes.columns) - 44]) + [f'tag_{i}' for i in range(1, 45)]

recipes = recipes.drop(columns=['tags_vector'])

recipes = recipes.drop_duplicates().reset_index(drop=True)

knn = NearestNeighbors(n_neighbors=10, algorithm='auto', metric='euclidean')

knn.fit(recipes.drop(columns=['recipe_id']))

def get_candidate_recipes(recipe_id):
    if recipe_id not in recipes['recipe_id'].values:
        print("Рецепт не найден")
        return []

    recipe_index = recipes[recipes['recipe_id'] == recipe_id].index[0]
    distances, indices = knn.kneighbors([recipes.drop(columns=['recipe_id']).iloc[recipe_index]], n_neighbors=10)
    candidates = recipes.iloc[indices[0][1:]]['recipe_id'].tolist()
    return candidates

In [11]:
import lightgbm as lgb
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.preprocessing import LabelEncoder


def process_features(interactions):
    def fill_empty_strings(column):
        return column.replace('', 'missing').fillna('missing')

    interactions['name_tokens'] = fill_empty_strings(interactions['name_tokens'])
    interactions['steps_tokens'] = fill_empty_strings(interactions['steps_tokens'])
    interactions['techniques'] = fill_empty_strings(interactions['techniques'])

    def ensure_list(arr):
        if isinstance(arr, list):
            return arr
        else:
            try:
                return eval(arr) if isinstance(arr, str) else [arr]
            except:
                return []

    for column in ['name_tokens', 'steps_tokens', 'techniques']:
        interactions[column] = interactions[column].apply(ensure_list)

    def extract_statistics(arr):
        arr = np.array(arr)
        return pd.Series({
            'mean': np.mean(arr) if len(arr) > 0 else 0,
            'std': np.std(arr) if len(arr) > 0 else 0,
            'min': np.min(arr) if len(arr) > 0 else 0,
            'max': np.max(arr) if len(arr) > 0 else 0,
            'sum': np.sum(arr) if len(arr) > 0 else 0
        })

    name_stats = interactions['name_tokens'].apply(extract_statistics)
    steps_stats = interactions['steps_tokens'].apply(extract_statistics)
    techniques_stats = interactions['techniques'].apply(extract_statistics)

    processed_interactions = pd.concat(
        [interactions.drop(['name_tokens', 'steps_tokens', 'techniques'], axis=1),
         name_stats.add_prefix('name_token_'),
         steps_stats.add_prefix('steps_token_'),
         techniques_stats.add_prefix('technique_')],
        axis=1
    )

    label_enc = LabelEncoder()
    processed_interactions['ingredient_ids'] = label_enc.fit_transform(processed_interactions['recipe_id'].astype(str))
    processed_interactions['tags_vector'] = label_enc.fit_transform(processed_interactions['recipe_id'].astype(str))

    return processed_interactions



def train_model(interactions):
    sampled_interactions = interactions.sample(frac=0.1, random_state=42)

    training_data = process_features(sampled_interactions)

    feature_cols = ['minutes', 'n_steps', 'n_ingredients',
                    'calorie_level', 'ingredient_ids', 'tags_vector',
                    'tag_1', 'tag_2', 'tag_3', 'tag_4', 'tag_5',
                    'tag_6', 'tag_7', 'tag_8', 'tag_9', 'tag_10',
                    'tag_11', 'tag_12', 'tag_13', 'tag_14',
                    'tag_15', 'tag_16', 'tag_17', 'tag_18',
                    'tag_19', 'tag_20', 'tag_21', 'tag_22',
                    'tag_23', 'tag_24', 'tag_25', 'tag_26',
                    'tag_27', 'tag_28', 'tag_29', 'tag_30',
                    'tag_31', 'tag_32', 'tag_33', 'tag_34',
                    'tag_35', 'tag_36', 'tag_37', 'tag_38',
                    'tag_39', 'tag_40', 'tag_41', 'tag_42',
                    'tag_43', 'tag_44',
                    'name_token_mean', 'name_token_std', 'name_token_min',
                    'name_token_max', 'name_token_sum',
                    'steps_token_mean', 'steps_token_std',
                    'steps_token_min', 'steps_token_max', 'steps_token_sum',
                    'technique_mean', 'technique_std', 'technique_min',
                    'technique_max', 'technique_sum']

    X = training_data[feature_cols].copy()
    print(X.columns)
    y = training_data['rating']

    for col in feature_cols:
        if X[col].dtype == 'object':
            X[col] = X[col].astype(str)

    lgb_train = lgb.Dataset(X, y)

    params = {
        'objective': 'binary',
        'metric': 'binary_logloss',
        'boosting_type': 'gbdt',
        'learning_rate': 0.05,
        'num_leaves': 31,
        'feature_fraction': 0.9,
        'bagging_fraction': 0.8,
        'bagging_freq': 5,
        'verbose': 0
    }

    tqdm_callback = lgb.callback.log_evaluation(period=10)
    model = lgb.train(params, lgb_train, num_boost_round=100, callbacks=[tqdm_callback])

    print('Model is trained')

    return model


def recommend_recipes(user_id, interactions):
    user_interactions = interactions[interactions['user_id'] == user_id]
    user_interactions = process_features(user_interactions)

    unique_recipe_ids = user_interactions['recipe_id'].unique()

    candidates = []

    for recipe_id in tqdm(unique_recipe_ids, desc="Finding similar recipes"):
        similar_recipes = get_candidate_recipes(recipe_id)
        for id in similar_recipes:
            candidate = user_interactions[user_interactions['recipe_id'] == recipe_id].copy()
            candidate['recipe_id'] = id
            candidates.append(candidate)

    if not candidates:
        return []

    candidates_df = pd.concat(candidates, ignore_index=True)

    feature_cols = [
       'minutes', 'n_steps', 'n_ingredients', 'calorie_level',
       'ingredient_ids', 'tags_vector', 'tag_1', 'tag_2', 'tag_3', 'tag_4',
       'tag_5', 'tag_6', 'tag_7', 'tag_8', 'tag_9', 'tag_10', 'tag_11',
       'tag_12', 'tag_13', 'tag_14', 'tag_15', 'tag_16', 'tag_17', 'tag_18',
       'tag_19', 'tag_20', 'tag_21', 'tag_22', 'tag_23', 'tag_24', 'tag_25',
       'tag_26', 'tag_27', 'tag_28', 'tag_29', 'tag_30', 'tag_31', 'tag_32',
       'tag_33', 'tag_34', 'tag_35', 'tag_36', 'tag_37', 'tag_38', 'tag_39',
       'tag_40', 'tag_41', 'tag_42', 'tag_43', 'tag_44', 'name_token_mean',
       'name_token_std', 'name_token_min', 'name_token_max', 'name_token_sum',
       'steps_token_mean', 'steps_token_std', 'steps_token_min',
       'steps_token_max', 'steps_token_sum', 'technique_mean', 'technique_std',
       'technique_min', 'technique_max', 'technique_sum'
    ]

    X_candidates = candidates_df[feature_cols]

    candidates_df['predicted_score'] = model.predict(X_candidates)

    recommended_recipes = candidates_df.sort_values(by='predicted_score', ascending=False).drop_duplicates('recipe_id').head(3)

    return recommended_recipes['recipe_id'].tolist()


model = train_model(interactions)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Index(['minutes', 'n_steps', 'n_ingredients', 'calorie_level',
       'ingredient_ids', 'tags_vector', 'tag_1', 'tag_2', 'tag_3', 'tag_4',
       'tag_5', 'tag_6', 'tag_7', 'tag_8', 'tag_9', 'tag_10', 'tag_11',
       'tag_12', 'tag_13', 'tag_14', 'tag_15', 'tag_16', 'tag_17', 'tag_18',
       'tag_19', 'tag_20', 'tag_21', 'tag_22', 'tag_23', 'tag_24', 'tag_25',
       'tag_26', 'tag_27', 'tag_28', 'tag_29', 'tag_30', 'tag_31', 'tag_32',
       'tag_33', 'tag_34', 'tag_35', 'tag_36', 'tag_37', 'tag_38', 'tag_39',
       'tag_40', 'tag_41', 'tag_42', 'tag_43', 'tag_44', 'name_token_mean',
       'name_token_std', 'name_token_min', 'name_token_max', 'name_token_sum',
       'steps_token_mean', 'steps_token_std', 'steps_token_min',
       'steps_token_max', 'steps_token_sum', 'technique_mean', 'technique_std',
       'technique_min', 'technique_max', 'technique_sum'],
      dtype='object')
Model is trained


In [12]:
user_id = 101053
recommended_recipes = recommend_recipes(user_id, interactions)

print(f"Recommended recipes for user {user_id}: {recommended_recipes}")

<ipython-input-11-470f1cf86aac>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interactions['name_tokens'] = fill_empty_strings(interactions['name_tokens'])
<ipython-input-11-470f1cf86aac>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  interactions['steps_tokens'] = fill_empty_strings(interactions['steps_tokens'])
<ipython-input-11-470f1cf86aac>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

Recommended recipes for user 101053: [27591, 108349, 298845]
